In [3]:
# 🔵 1. MNIST Digit Detection using TensorFlow (Low-Level API, FIXED)
import tensorflow as tf
from tensorflow.keras.datasets import mnist as tf_mnist

# 🔹 Data Preparation
(x_train_tf, y_train_tf), (x_test_tf, y_test_tf) = tf_mnist.load_data()
x_train_tf, x_test_tf = x_train_tf / 255.0, x_test_tf / 255.0

train_ds_tf = tf.data.Dataset.from_tensor_slices((x_train_tf, y_train_tf)).shuffle(10000).batch(32)
test_ds_tf = tf.data.Dataset.from_tensor_slices((x_test_tf, y_test_tf)).batch(32)

# 🔹 Fixed Model Definition using tf.keras.Model
class TFModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10)

    def call(self, x):
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

model_tf = TFModel()
loss_fn_tf = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer_tf = tf.keras.optimizers.Adam()

# 🔹 Training
@tf.function
def train_step_tf(images, labels):
    with tf.GradientTape() as tape:
        logits = model_tf(images)
        loss = loss_fn_tf(labels, logits)
    grads = tape.gradient(loss, model_tf.trainable_variables)
    optimizer_tf.apply_gradients(zip(grads, model_tf.trainable_variables))
    return loss

for epoch in range(10):
    for images, labels in train_ds_tf:
        loss = train_step_tf(images, labels)
    print(f"🔵 TF Epoch {epoch+1}, Loss: {loss:.4f}")

# 🔹 Evaluation
correct_tf = 0
total_tf = 0
for images, labels in test_ds_tf:
    logits = model_tf(images)
    predictions = tf.argmax(logits, axis=1)
    correct_tf += tf.reduce_sum(tf.cast(predictions == tf.cast(labels, tf.int64), tf.float32))
    total_tf += images.shape[0]

print(f"🔵 TF Test Accuracy: {correct_tf / total_tf:.4f}")


🔵 TF Epoch 1, Loss: 0.1545
🔵 TF Epoch 2, Loss: 0.0219
🔵 TF Epoch 3, Loss: 0.0147
🔵 TF Epoch 4, Loss: 0.0118
🔵 TF Epoch 5, Loss: 0.0335
🔵 TF Epoch 6, Loss: 0.0468
🔵 TF Epoch 7, Loss: 0.0140
🔵 TF Epoch 8, Loss: 0.0201
🔵 TF Epoch 9, Loss: 0.0071
🔵 TF Epoch 10, Loss: 0.0245
🔵 TF Test Accuracy: 0.9783


In [4]:
# 🔵 3. MNIST Digit Detection using Keras (TensorFlow Backend)
from tensorflow.keras.datasets import mnist as keras_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

# 🔹 Data Preparation
(x_train_kr, y_train_kr), (x_test_kr, y_test_kr) = keras_mnist.load_data()
x_train_kr, x_test_kr = x_train_kr / 255.0, x_test_kr / 255.0
y_train_kr = to_categorical(y_train_kr)
y_test_kr = to_categorical(y_test_kr)

# 🔹 Model Definition
model_kr = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# 🔹 Compilation
model_kr.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 🔹 Training
model_kr.fit(x_train_kr, y_train_kr, batch_size=32, epochs=5, validation_split=0.2)

# 🔹 Evaluation
loss_kr, acc_kr = model_kr.evaluate(x_test_kr, y_test_kr)
print(f"🔵 Keras Test Loss: {loss_kr:.4f}")
print(f"🔵 Keras Test Accuracy: {acc_kr:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.8685 - loss: 0.4718 - val_accuracy: 0.9498 - val_loss: 0.1684
Epoch 2/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.9593 - loss: 0.1387 - val_accuracy: 0.9643 - val_loss: 0.1200
Epoch 3/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9724 - loss: 0.0905 - val_accuracy: 0.9684 - val_loss: 0.1049
Epoch 4/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - accuracy: 0.9799 - loss: 0.0660 - val_accuracy: 0.9712 - val_loss: 0.0949
Epoch 5/5
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - accuracy: 0.9855 - loss: 0.0512 - val_accuracy: 0.9732 - val_loss: 0.0891
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9705 - loss: 0.0944
🔵 Keras Test Loss: 0.0833
🔵 Keras Test Accuracy: 0.9736


In [5]:
# 🔵 2. MNIST Digit Detection using PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 🔹 Data Preparation
transform_pt = transforms.Compose([transforms.ToTensor()])
train_data_pt = datasets.MNIST(root='./data', train=True, download=True, transform=transform_pt)
test_data_pt = datasets.MNIST(root='./data', train=False, download=True, transform=transform_pt)
train_loader_pt = DataLoader(train_data_pt, batch_size=32, shuffle=True)
test_loader_pt = DataLoader(test_data_pt, batch_size=32, shuffle=False)

# 🔹 Model Definition
class NetPT(nn.Module):
    def __init__(self):
        super(NetPT, self).__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

model_pt = NetPT()
criterion_pt = nn.CrossEntropyLoss()
optimizer_pt = optim.Adam(model_pt.parameters(), lr=0.001)

# 🔹 Training
for epoch in range(5):
    for images, labels in train_loader_pt:
        outputs = model_pt(images)
        loss = criterion_pt(outputs, labels)
        optimizer_pt.zero_grad()
        loss.backward()
        optimizer_pt.step()
    print(f"🔵 PT Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 🔹 Evaluation
correct_pt = 0
total_pt = 0
with torch.no_grad():
    for images, labels in test_loader_pt:
        outputs = model_pt(images)
        _, predicted = torch.max(outputs, 1)
        total_pt += labels.size(0)
        correct_pt += (predicted == labels).sum().item()

print(f"🔵 PT Test Accuracy: {100 * correct_pt / total_pt:.2f}%")


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 478kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.44MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.01MB/s]


🔵 PT Epoch 1, Loss: 0.2350
🔵 PT Epoch 2, Loss: 0.0984
🔵 PT Epoch 3, Loss: 0.0433
🔵 PT Epoch 4, Loss: 0.0140
🔵 PT Epoch 5, Loss: 0.0363
🔵 PT Test Accuracy: 97.59%
